In [1]:
import yfinance as yf
import pandas as pd

In [2]:
tickerssectoresdic = {
  "Energy": ["XLE", "USO", "UNG"],
  "Materials": ["XLB", "PDBC", "GLD"],
  "Industrials": ["XLI", "IYT", "IVZ"],
  "Consumer Discretionary": ["XLY", "XRT", "PX"],
  "Consumer Staples": ["XLP", "KXI", "RYH"],
  "Health Care": ["XLV", "PFE", "IBB"],
  "Financials": ["XLF", "KRE", "XLF"],
  "Information Technology": ["XLK", "SOXX", "FTEC"],
  "Communication Services": ["XLC", "IGV", "JCOM"],
  "Utilities": ["XLU", "VPU", "WEN"]}

In [3]:
sectores = list(tickerssectoresdic.keys())



In [4]:
tickers_list = [ticker for tickers in tickerssectoresdic.values() for ticker in tickers]


In [5]:
# Diccionario para almacenar los DataFrames de cada sector y ticker
dataframes = {}

# Bucle para crear DataFrames para cada sector y ticker
for sector, tickers in tickerssectoresdic.items():
    dataframes[sector] = {}
    for ticker in tickers:
        etf_df = yf.download(ticker, start="2005-01-01")
        dataframes[sector][ticker] = etf_df
        # Haz algo con el DataFrame para cada ticker, por ejemplo, imprimir la información básica
        print(f"Información básica para el ticker {ticker} del sector {sector}:")
        print(etf_df.info())
        print("-----------------------------------")

[*********************100%***********************]  1 of 1 completed
Información básica para el ticker XLE del sector Energy:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4620 entries, 2005-01-03 to 2023-05-10
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       4620 non-null   float64
 1   High       4620 non-null   float64
 2   Low        4620 non-null   float64
 3   Close      4620 non-null   float64
 4   Adj Close  4620 non-null   float64
 5   Volume     4620 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 252.7 KB
None
-----------------------------------
[*********************100%***********************]  1 of 1 completed
Información básica para el ticker USO del sector Energy:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4301 entries, 2006-04-10 to 2023-05-10
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Op

In [6]:
fechas = {
    'Mercado_alcista_2005-2007': ['2005-01-01', '2007-10-31'],
    'Crisis_financiera_2007-2009': ['2007-11-01', '2009-03-09'],
    'Recuperación_2009-2012': ['2009-03-10', '2012-12-31'],
    'Expansión_2013-2020': ['2013-01-01', '2020-02-19'],
    'Pandemia_bajista_2020': ['2020-02-20', '2020-03-23'],
    'Pandemia_alcista_2020-2021': ['2020-03-24', '2021-12-31'],
    "Bajista": ['2022-01-01', '2023-05-10']}

In [33]:
# Convertir las fechas en el diccionario al formato adecuado
#for fecha_key in fechas.keys():
 #   fechas[fecha_key][0] = pd.to_datetime(fechas[fecha_key][0])
  #  fechas[fecha_key][1] = pd.to_datetime(fechas[fecha_key][1])

# Bucle para iterar por cada sector del DataFrame obtenido en el primer bucle
#for sector in sectores:
 #   sector_data = dataframes[sector]
  #  tickers_list = list(dataframes[sector].keys())
    
   # for ticker in tickers_list:
    #    sector_data_ticker = dataframes[sector][ticker]
     #   ma_cal (sector_data_ticker,100,50)
    # Dividir el DataFrame de cada sector según las fechas y guardar en archivos CSV
      #  for fecha_key, fecha_value in fechas.items():
       #     dataframe = sector_data_ticker[(sector_data_ticker.index >= fecha_value[0]) & (sector_data_ticker.index <= fecha_value[1])]
        #    dataframe.to_csv(f'dataframe_{sector}_{ticker}_{fecha_key.replace(" ", "_")}.csv', index=False)


In [17]:
def ma_cal (df, slow, fast):
    df['slow'] = df['Close'].rolling(slow).mean()
    df['fast'] = df['Close'].rolling(fast).mean()
    
def vectorized ( df, slow, fast):
    ma_cal (df, slow, fast)
    df= df.fillna(0)
    if df["slow"].empty:
        return None
    first_buy = pd.Series (df.index == (df.slow < df.fast).idxmax(), index=df.index)
    real_signal = first_buy | (df.slow < df.fast).diff()
    trades = df[real_signal]
    if len (trades) % 2 !=0:
        mtm = df.tail(1).copy()
        mtm.price = mtm.Close
        trades =pd.concat([trades,mtm])
    profits = trades.price.diff()[1::2]/ trades.price[0::2].values
    gain = (profits + 1).prod()
    return gain

def backtest(df, slow, fast):
    
    in_position = False
    profits = []
    
    for index, row in df.iterrows():
        if not in_position:
            if row.slow < row.fast:
                buyprice = row.price
                in_position = True
        if in_position:
                if row.slow > row.fast:
                    profit = (row.price - buyprice)/buyprice
                    profits.append(profit)
                    in_position = False
                    
    if in_position:
        profit =(row.Close - buyprice)/buyprice
        profits.append(profit)
    gain = (pd.Series(profits) + 1).prod()
    
    return gain

In [14]:
# Convertir las fechas en el diccionario al formato adecuado
results = []
ticker_evaluado = []
for fecha_key in fechas.keys():
    fechas[fecha_key][0] = pd.to_datetime(fechas[fecha_key][0])
    fechas[fecha_key][1] = pd.to_datetime(fechas[fecha_key][1])

# Bucle para iterar por cada sector del DataFrame obtenido en el primer bucle
for sector in sectores:
    sector_data = dataframes[sector]
    tickers_list = list(dataframes[sector].keys())
    
    for ticker in tickers_list:
        sector_data_ticker = dataframes[sector][ticker]
        sector_data_ticker = sector_data_ticker.fillna(0)
        sector_data_ticker_1 = sector_data_ticker.copy()
        sector_data_ticker_1.loc[:,"price"] = sector_data_ticker_1.Open.shift(-1)
    # Dividir el DataFrame de cada sector según las fechas y guardar en archivos CSV
        for fecha_key, fecha_value in fechas.items():
            dataframe = sector_data_ticker_1[(sector_data_ticker_1.index >= fecha_value[0]) & (sector_data_ticker_1.index <= fecha_value[1])]
            print("result for" + ticker)
            print(vectorized (dataframe, 1,2))
            results.append(vectorized(dataframe,1,2))
            stoc = ticker + fecha_key
            ticker_evaluado.append (stoc)
            

result forXLE


KeyError: 'slow'

In [51]:
# Convertir las fechas en el diccionario al formato adecuado
results = []
ticker_evaluado = []
duracion = []
additional_calculation = []

for fecha_key in fechas.keys():
    fechas[fecha_key][0] = pd.to_datetime(fechas[fecha_key][0])
    fechas[fecha_key][1] = pd.to_datetime(fechas[fecha_key][1])

# Bucle para iterar por cada sector del DataFrame obtenido en el primer bucle
for sector in sectores:
    sector_data = dataframes[sector]
    tickers_list = list(dataframes[sector].keys())
    
    for ticker in tickers_list:
        sector_data_ticker = dataframes[sector][ticker]
        sector_data_ticker = sector_data_ticker.fillna(0)
        sector_data_ticker_1 = sector_data_ticker.copy()
        sector_data_ticker_1.loc[:,"price"] = sector_data_ticker_1.Open.shift(-1)
        ma_cal(sector_data_ticker_1, slow=14, fast=7)

        for fecha_key, fecha_value in fechas.items():
            dataframe = sector_data_ticker_1[(sector_data_ticker_1.index >= fecha_value[0]) & (sector_data_ticker_1.index <= fecha_value[1])]
            results.append(backtest(dataframe,10,5))
            stoc = ticker + fecha_key
            ticker_evaluado.append (stoc)
            duration = (fecha_value[1] - fecha_value[0]).days + 1
            duracion.append(duration)
            cumulative_return = (dataframe.Close.pct_change() + 1).prod()
            additional_calculation.append(cumulative_return)


C:\Temp\ipykernel_2628\2719541465.py:40: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gain = (pd.Series(profits) + 1).prod()
C:\Temp\ipykernel_2628\2719541465.py:40: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gain = (pd.Series(profits) + 1).prod()
C:\Temp\ipykernel_2628\2719541465.py:40: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gain = (pd.Series(profits) + 1).prod()
C:\Temp\ipykernel_2628\2719541465.py:40: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gain = (pd.Series(profits) + 1).prod()
C:\Temp\ipykernel_2628\27195

In [52]:
profits = pd.DataFrame({"profit":results,"len":duracion,"cumulative_return": additional_calculation}, index = ticker_evaluado)
profits["profit_len"] = profits.profit / profits.len
profits["dif_esta_long"] = profits.profit - profits.cumulative_return

In [50]:
profits.to_excel("profits_etfs_diferentesescenarios.xlsx")

In [53]:
profits.sort_values(by="profit",ascending=False)

,profit,len,cumulative_return,profit_len,dif_esta_long
SOXXExpansión_2013-2020,2.516747,2606,4.947524,0.000966,-2.430777
XRTPandemia_alcista_2020-2021,2.416444,648,3.035979,0.003729,-0.619535
WENExpansión_2013-2020,2.292536,2606,4.959575,0.000880,-2.667039
PFEPandemia_alcista_2020-2021,2.021051,648,2.095579,0.003119,-0.074528
IBBRecuperación_2009-2012,1.894051,1393,2.239230,0.001360,-0.345179
...,...,...,...,...,...
SOXXCrisis_financiera_2007-2009,0.647587,495,0.409877,0.001308,0.237710
XLFCrisis_financiera_2007-2009,0.493594,495,0.195625,0.000997,0.297969
UNGExpansión_2013-2020,0.437863,2606,0.209430,0.000168,0.228433
IVZCrisis_financiera_2007-2009,0.396374,495,0.322482,0.000801,0.073892


In [54]:
profits.sort_values(by="dif_esta_long",ascending=False)


,profit,len,cumulative_return,profit_len,dif_esta_long
USOExpansión_2013-2020,1.118349,2606,0.330574,0.000429,0.787775
UNGCrisis_financiera_2007-2009,0.920419,495,0.358486,0.001859,0.561932
XLEPandemia_bajista_2020,0.954118,33,0.430188,0.028913,0.523930
IVZPandemia_bajista_2020,0.939072,33,0.422074,0.028457,0.516998
USOCrisis_financiera_2007-2009,0.889026,495,0.396979,0.001796,0.492046
...,...,...,...,...,...
FTECExpansión_2013-2020,1.378141,2606,3.225858,0.000529,-1.847717
XLKExpansión_2013-2020,1.545941,2606,3.448172,0.000593,-1.902230
SOXXExpansión_2013-2020,2.516747,2606,4.947524,0.000966,-2.430777
IGVExpansión_2013-2020,1.463098,2606,4.106406,0.000561,-2.643308
